# 02c – 4. Experiment: Modell-Vergleich: Whisper Large-v3 (Audio-Only) vs. AV-HuBERT

## Motivation

Nachdem Fine-Tuning-Ansätze (02a, 02b) keine Verbesserung gegenüber BL4 brachten,
wird hier ein grundlegend anderes Modell getestet: **Whisper Large-v3** als
reines Audio-ASR-System (kein visueller Input).

**Hypothese:** Whisper ist ein state-of-the-art generisches ASR-Modell. Kann es
trotz fehlendem visuellem Input mit BL4 mithalten?

## Ergebnis (Vorschau)

| Modell | WER | Joint Error |
|--------|-----|-------------|
| Whisper Large-v3 | ~1.03 | ~0.59 |
| BL4 (beam=12, len=20) | ~0.495 | ~0.324 |

Whisper ist **~52 % schlechter** in WER und ~45 % schlechter in Joint Error.
In der überlappenden Mehrsprecher-Umgebung von MCoRec ist visueller Input entscheidend.

**Hinweis zum Bugfix:** Dieser Lauf wurde **vor dem Bugfix** in `segmentation.py` durchgeführt (`min_duration_off` las fälschlicherweise den Wert von `min_duration_on`). Das ist **gewollt**: Der Bugfix wurde erst nach Abschluss der LLM- und Hyperparameter-Experimente entdeckt. Da der Bugfix allein die WER zunächst verschlechterte, wurde erst in `02j_`/`02k_` die Kombination aus Bugfix + `min_duration`-Optimierung erarbeitet, die schließlich das beste Ergebnis lieferte.

## 1 – Setup: Arbeitsverzeichnis & Imports

In [1]:
import os, sys
import pandas as pd

# Arbeitsverzeichnis auf Repo-Root setzen (Voraussetzung für alle relativen Pfade)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, run_eval_and_log, append_eval_results_for_experiments

## 2 – Modell-Definition

`whisper_audio` ist eine in `inference.py` ergänzte Modellklasse, die
Whisper rein audio-basiert betreibt – ohne Lip-Crop-Input.
Der Checkpoint-Name wird direkt an die openai-whisper-Bibliothek übergeben.

In [2]:
MODELS = {
    "whisper_large_v3": {
        "model_type": "whisper_audio",
        # Checkpoint-Name gemäß openai-whisper-Konvention:
        # Optionen: "tiny", "base", "small", "medium", "large-v2", "large-v3"
        "chkpt": "large-v3"}
}

## 3 – Sessions & Experimente

Gleiches 5-Session-Subset für direkte Vergleichbarkeit mit allen vorherigen Experimenten.

In [3]:
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

In [4]:
EXPERIMENTS = {
    # E16/E17: Whisper mit den bisher besten Hyperparametern (aus 02_)
    # um einen fairen Vergleich mit BL4 bei gleichen Decoding-Parametern zu ermöglichen
    "E16_whisper_bs8_len20": {
    "base_model": "whisper_large_v3",
    "beam_size": 8,
    "max_length": 20,
    "comment": "Whisper Large-v3, beam=8, len=20",
    },
    
    "E17_whisper_bs12_len20": {
    "base_model": "whisper_large_v3",
    "beam_size": 12,
    "max_length": 20,
    "comment": "Whisper Large-v3, beam=12, len=20",
    },
}


## 4 – Inference

2 Experimente × 5 Sessions = 10 Läufe.

In [5]:
for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Experimente für {sid} ##########")

    for exp_name in EXPERIMENTS:
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )


########## Starte Experimente für session_40 ##########

Starte Inference für Experiment: E16_whisper_bs8_len20
  base_model      = whisper_large_v3
  model_type      = whisper_audio
  checkpoint_path = large-v3
  beam_size       = 8
  max_length      = 20
  output_dir_name = output_E16_whisper_bs8_len20
  session_dir     = data-bin/dev_without_central_videos/dev/session_40
  comment         = Whisper Large-v3, beam=8, len=20
Loading whisper_audio model...
Loading Whisper model 'large-v3' on device cuda


100%|█████████████████████████████████████| 2.88G/2.88G [00:55<00:00, 56.0MiB/s]


whisper_audio model loaded successfully!
Inferring 1 sessions using whisper_audio model
Processing session session_40


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [01:17<06:28, 77.75s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 26/26 [02:01<00:00,  4.65s/it]

Processing speakers:  20%|██        | 1/5 [02:38<10:35, 158.84s/it]


Processing speakers:  40%|████      | 2/5 [05:25<08:09, 163.32s/it]


Processing speakers:  60%|██████    | 3/5 [08:10<05:28, 164.09s/it]


Processing speakers:  80%|████████  | 4/5 [12:06<03:12, 192.46s/it]


Processing speakers: 100%|██████████| 5/5 [14:29<00:00, 173.92s/it]


## 5 – Evaluation & Aggregation

In [6]:
# Ergebnisse auswerten und an gemeinsame CSV anhängen
df_dev = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSION_IDS,
    target_csv="results_dev_subset_by_session.csv",
)



########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clusterin

## 6 – Vergleich: Whisper vs. BL4

Whisper-Ergebnisse (E16, E17) werden den besten BL4-Konfigurationen (E08, E09) gegenübergestellt.

In [7]:
import pandas as pd

dev_df = pd.read_csv("results_dev_subset_by_session.csv")

# Whisper-Ergebnisse über alle Sessions mitteln
whisper_models = ["E16_whisper_bs8_len20", "E17_whisper_bs12_len20"]
whisper_df = (
    dev_df[dev_df["model"].isin(whisper_models)]
    .groupby("model")[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

# BL4-Referenz: beste Konfigurationen aus 02_ (beam=8/12, len=20)
best_avsr_models = ["E08_bs8_len20", "E09_bs12_len20"]
avsr_df = (
    dev_df[dev_df["model"].isin(best_avsr_models)]
    .groupby("model")[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

print("Whisper:")
display(whisper_df)
print("AVSR finetuned:")
display(avsr_df)


Whisper:


,model,avg_speaker_wer,avg_joint_error
0,E16_whisper_bs8_len20,1.031613,0.591999
1,E17_whisper_bs12_len20,1.025096,0.588740


AVSR finetuned:


,model,avg_speaker_wer,avg_joint_error
0,E08_bs8_len20,0.495798,0.324091
1,E09_bs12_len20,0.495416,0.323900


## 7 – Interpretation

| Modell | WER | Joint Error | rel. Δ WER vs. BL4 |
|--------|-----|-------------|--------------------|
| Whisper Large-v3 (beam=8/12) | ~1.03 | ~0.59 | +~108 % |
| BL4 (beam=12, len=20) | ~0.495 | ~0.324 | – |

Whisper ist mit einer WER von 1.03 mehr als doppelt so fehleranfällig wie BL4.
Auch die Joint Error Rate (0.59 vs. 0.324) liegt deutlich höher.

**Warum ist Whisper so viel schlechter?**

1. **Kein visueller Input:** In der überlappenden Mehrsprecher-Umgebung von MCoRec
   liefern Lip-Crops pro Sprecher entscheidende Kontextinformation, die Whisper fehlt.
2. **Kein In-Domain-Fine-Tuning:** Whisper wurde auf generischen Audiodaten trainiert,
   nicht auf MCoRec-spezifischen Sprachmustern und Vokabular.
3. **Fehlende Sprecher-Segmentierung:** Ohne sprecher-spezifische Eingabe produziert
   Whisper viele Insertions und inhaltlich abweichende Wörter, was WER und JER stark erhöht.

**Schlussfolgerung:** Audio-only ASR ist für die MCoRec-Challenge ungeeignet.
Der audio-visuelle Ansatz von BL4 bleibt die Basis für alle weiteren Experimente.
